In [2]:
import numpy as np
import pandas as pd
from scipy import stats

import pyreto
from pyreto.tests import utilities

<h2> Pareto distributions </h2>

In [30]:
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [28]:
my = Pareto(result1.params['alpha'], 1.0)

In [29]:
my.pdf(1.0)

TypeError: unorderable types: float() >= pareto_gen()

In [14]:
prng = np.random.RandomState(42)
alpha, xmin = 3.0, 1.0
pareto_data = pd.Series(pyreto.distributions.Pareto._rvs(prng, 2500, xmin, alpha), name='samples')

In [15]:
# check that I get same estimate for alpha given reported xmin...
result1 = pyreto.distributions.Pareto.fit(pareto_data, xmin=xmin)

In [16]:
result1.params['alpha']

2.9998119111215154

In [7]:
result1.xmin

1.0

In [8]:
f = p(result1.params['alpha'], 1.0)

In [9]:
f.pdf(5.0)

0.016003339044528487

In [10]:
pyreto.distributions.Pareto._pdf(5.0, 1.0, result1.params['alpha'])

0.016003339044528487

In [12]:
f = stats.pareto(2.0, 1.0)

In [13]:
stats.pareto.

In [28]:
pyreto.distributions.Pareto._pdf??

In [22]:
# check that I get same estimate for alpha given reported xmin...
result2 = pyreto.distributions.Pareto.fit(pareto_data, xmin=None, quantile=0.9, method='brute')

In [23]:
result2.params['alpha']

1.9909069323487545

In [24]:
result2.xmin

1.1331781782153696

In [25]:
# check that I get same estimate for alpha given reported xmin...
result3 = pyreto.distributions.Pareto.fit(pareto_data, xmin=None, quantile=0.9, method='bounded')

In [26]:
result3.params['alpha']

1.9924312888829019

In [27]:
result3.xmin

1.3693896782872266

<h2> Cauchy distribution </h2>

In [3]:
prng = np.random.RandomState(1234)
cauchy_rvs = stats.cauchy.rvs(size=2500, random_state=prng)
cauchy_data = pd.Series(cauchy_rvs, name='samples')

In [4]:
# check that I get same estimate for alpha given reported xmin...
result = pyreto.distributions.Pareto.fit(cauchy_data, scale=None, quantile=0.99, method='brute')

In [5]:
# will raise assertion error if test fails!
desired_alpha = 2.0
utilities.test_scaling_exponent_estimation(desired_alpha, result)

<h2> Levy-Stable distributions </h2>

In [15]:
prng = np.random.RandomState(4321)
desired_alpha = stats.uniform.rvs(1, 3, random_state=prng)
beta = 1.0 
stable_rvs = stats.levy_stable.rvs(desired_alpha - 1, beta, size=1000, random_state=prng)
stable_data = pd.Series(stable_rvs, name='samples')

In [16]:
result = pyreto.distributions.Pareto.fit(stable_data, scale=None, quantile=0.99, method='bounded')

In [17]:
# will raise assertion error if test fails!
utilities.test_scaling_exponent_estimation(desired_alpha, result)

In [18]:
# goodness of fit test should fail to reject Pareto tails!
test_kwargs = {'seed': None, 'result': result, 'data': stable_data,
               'scale': None, 'quantile': 0.99, 'method': 'bounded'}
pvalue, _ = pyreto.distributions.Pareto.test_goodness_of_fit(**test_kwargs)
assert pvalue > 0.1, "Goodness of fit test Type I error!"

In [12]:
pvalue

0.088861506599237791

In [14]:
result.params

{'b': 0.68630667165978276, 'loc': 0, 'scale': 467.86812683010794}